In [ ]:
%matplotlib inline

# 训练分类器

你已经知道怎么定义神经网络、计算损失并更新网络权重了。

现在你可能会想，

## 那数据呢？

通常来说，当你需要处理图像、文本、音频或视频数据的时候，你可以使用把数据加载到 numpy array 的标准 Python 包。
然后你就可以把这个 array 转换为 `torch.*Tensor` 了。

- 对于图像，像 Pillow 和 OpenCV 这样的包是可以的
- 对于音频，像 scipy 和 librosa 这样的包是可以的
- 对于文本，基于 Python 或 Cython 的，或者 NLTK 和 SpaCy 都是可以的

我们针对视觉场景构建了一个名叫 `torchvision` 的包，它包含用于 Imagenet 、 CIFAR10 、 MNIST 等常见数据集的数据加载器，以及用于图像的数据转换器，即 `torchvision.datasets` 和 `torch.utils.data.DataLoader` 。

这提供了非常大的便利并且避免了编写重复代码。

在这篇教程中，我们将使用 CIFAR10 数据集。
它包含以下种类：飞机、汽车、鸟、猫、鹿、狗、青蛙、马、船、货车。 
CIFAR-10 中的图片尺寸是 3 x 32 x 32 的，也就是说，是 32 x 32 像素大小， 3 个色彩通道的彩色图片。

![cifar10](./img/cifar10.png)

## 训练一个图像分类器

我们将顺序进行如下步骤：

1. 使用 `torchvision` 加载并标准化 CIFAR10 的训练数据集和测试数据集
2. 定义一个卷积神经网络
3. 定义一个损失函数
4. 使用训练数据训练网络
5. 使用测试数据测试网络

### 1. 加载并标准化 CIFAR10

使用 `torchvision` 可以非常轻松地加载 CIFAR10 。


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

`torchvision` 数据集的输出是范围在 [0, 1] 的 PILImage 图像。
我们将它们转换为标准化到范围 [-1. 1] 的 Tensor 。


In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

为了更有意思，我们先展示一些训练图像。


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 显示一张图片的函数


def imshow(img):
    img = img / 2 + 0.5     # 非标准化
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# 获取一些随机训练图像
dataiter = iter(trainloader)
images, labels = dataiter.next()

# 显示图像
imshow(torchvision.utils.make_grid(images))
# 打印 label
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

### 2. 定义一个卷积神经网络
从之前讲神经网络的那一节把写好的代码复制过来，然后把它修改成接收 3 通道图像（替换掉原来定义的接收 1 通道图像那部分）。


In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

### 3. 定义一个损失函数和优化器
让我们使用分类交叉熵损失和带动量的SGD。


In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

### 4. 训练网络

从这里开始事情将变得有趣起来。
我们只需要循环遍历数据迭代器，然后将输入馈送给网络并进行优化。


In [ ]:
for epoch in range(2):  # 在数据集上循环迭代多次

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # 得到输入
        inputs, labels = data

        # 将参数梯度清零
        optimizer.zero_grad()

        # 前项 + 后项 + 优化
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 打印统计结果
        running_loss += loss.item()
        if i % 2000 == 1999:    # 每 2000 个小批次打印一次
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

### 5. 使用测试数据测试网络

我们已经使用训练数据集对网络进行了两次训练。
但是我们需要检查一下网络究竟有没有学习到东西。

我们将通过对比神经网络的预测输出种类标签（label）和图片的真实种类来检查这一点。
如果预测正确，我们将样本添加到正确预测的列表中。

好的，第一步。让我们显示一张测试集中的图像来熟悉测试集。


In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()

# 打印图像
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

好的，现在让我们看一看神经网络认为上面这些例子是什么：


In [ ]:
outputs = net(images)

输出是 10 个种类的能量。
一个种类的能量越高，神经网络越认为这幅图像属于这个种类。
所以，让我们取得能量最高的 index ：


In [ ]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

结果看起来非常不错。

让我们看一下神经网络在整个数据集上的表现怎么样。


In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

看起来结果比完全随机的情况好很多很多，因为完全随机的准确率应该是 10 % （从10个类别中随机选取一个类别）。
网络应该是学到了一些东西的。

看一下哪些是分类效果不错的类别，哪些是分类效果不是很好的类别：


In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

下一步做什么呢？

我们怎么在 GPU 上运行这个神经网络？

## 在 GPU 上训练
你可以像转移一个 Tensor 一样，将神经网络转移到 GPU 上。

首先如果 CUDA 可用的话，将我们的设备定义为第一个可见的 cuda 设备：


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 假设我们在一台 CUDA 设备上，这条语句就应该打印一个 CUDA 设备：

print(device)

这一节剩下的内容假定 `device` 是 CUDA 设备。

然后这些方法将递归遍历所有模块，并将它们的参数和缓存转换为 CUDA tensor 。

```python
    net.to(device)
```

记住，你每一步必须把输入和目标值（target）也送到 GPU：

```python
    inputs, labels = inputs.to(device), labels.to(device)
```

为什么我没有感觉到比使用 CPU 变快很多呢？因为你的网络太太太小了。

**练习：**尝试增加网络宽度（第一个 `nn.Conv2d` 的第二个参数和第二个 `nn.Conv2d` 的第一个参数 - 它们需要是想用的数字），看看你能得到多大的加速。

**达成目标：**

- 较高水平地理解 PyTorch 的 Tensor 库和神经网络
- 训练一个小神经网络来分类图像

## 在多个 GPU 上训练
如果你想看更多的使用 GPU 进行更快加速的内容，
请查看[可选内容：数据并行](./data_parallel_tutorial.ipynb)

## 下一步我可以看些什么？

- [训练神经网络让它学会玩游戏](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html)
- [在 imagenet 上训练一个最先进的 ResNet](https://github.com/pytorch/examples/tree/master/imagenet)
- [使用生成对抗网络训练一个人脸生成器](https://github.com/pytorch/examples/tree/master/dcgan)
- [使用 LSTM 网络训练一个词级语言模型](https://github.com/pytorch/examples/tree/master/word_language_model)
- [更多示例](https://github.com/pytorch/examples)
- [更多教程](https://github.com/pytorch/tutorials)
- [在论坛讨论 PyTorch](https://discuss.pytorch.org/)
- [在 Slack 上与其他用户聊天](https://pytorch.slack.com/messages/beginner/)
